In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import importlib
import plotly.express as px
from datetime import datetime

sys.path.append('../lib')
import utils
import pandas_utils
import pesys_utils

importlib.reload(utils)
importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

utils.set_secrets_as_env()

In [ ]:
con = pesys_utils.open_pen_connection()

In [ ]:
df_query = pandas_utils.pandas_from_sql(con=con, sqlfile='../sql/retur_arsak.sql', lowercase=True, tuning=10000)

In [ ]:
df = df_query.copy()
df["dato"] = pd.to_datetime(df.dato)
df.set_index("dato", inplace=True)

df_recent = df[df.index.year == 2023]
df_recent.head()

In [ ]:
df_sakstype = (
    df
    .groupby(["sakstype"])[["antall"]]
    .resample('MS')
    .sum()
    .reset_index()      
)
df_sakstype.head()

In [ ]:
df_arsak = (
    df
    .groupby(["retur_arsak"])[["antall"]]
    .resample('MS')
    .sum()
    .reset_index()      
)
df_sakstype.head()

In [ ]:
df_kravtype = (
    df
    .groupby(["sakstype", "kravtype"])[["antall"]]
    .resample('MS')
    .sum()
    .reset_index()      
)
df_kravtype.head()

In [ ]:
df.index.year.unique()

In [ ]:
px.line(df_sakstype, "dato", "antall", color="sakstype")

In [ ]:
px.line(df_arsak, "dato", "antall", color="retur_arsak")

In [ ]:
sakstyper = ['Alderspensjon', "Uføretrygd"]
figs = {}
for sakstype in sakstyper:
    temp_df = df_kravtype[df_kravtype.sakstype == sakstype]
    fig = px.line(temp_df, "dato", "antall", color="kravtype", title=sakstype)
    figs[f"kravtyper_" + sakstype] = fig

for _, fig in figs.items():
    fig.show()